<a href="https://colab.research.google.com/github/IM07813/Transformers/blob/main/basic_essay_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow_ranking
!pip install -q tensorflow_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.4/150.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 57.8 MB/s eta 0:00:00


In [9]:
import pandas as pd 
df = pd.read_csv("/content/drive/MyDrive/essay-scores.csv", encoding='latin1')
df.shape

(10692, 2)

##data cleaning using pandas

In [7]:

import pandas as pd


df = pd.read_csv("/content/drive/MyDrive/essay-scores.csv", encoding='latin1')


df = df[df["scores"] != 0]


df.to_csv("essay_dataset_modified.csv", index=False)

In [10]:
df2 = pd.read_csv("/content/essay_dataset_modified.csv", encoding='latin1')
df2.shape

(10225, 2)

In [12]:
df2.tail

<bound method NDFrame.tail of                                                    essay  scores
0      Dear local newspaper, I think effects computer...     4.0
1      Dear @CAPS1 @CAPS2, I believe that using compu...     4.0
2      Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...     3.0
3      Dear Local Newspaper, @CAPS1 I have found that...     5.0
4      Dear @LOCATION1, I know having computers has a...     4.0
...                                                  ...     ...
10220                                                NaN     NaN
10221                                                NaN     NaN
10222                                                NaN     NaN
10223                                                NaN     NaN
10224                                                NaN     NaN

[10225 rows x 2 columns]>

In [14]:
df3 = df2.dropna()

# Save the modified dataframe to a new csv file
df.to_csv("essay_dataset_modified_v2.csv", index=False)

In [15]:
df3.shape

(10218, 2)

In [16]:
df3.head

<bound method NDFrame.head of                                                    essay  scores
0      Dear local newspaper, I think effects computer...     4.0
1      Dear @CAPS1 @CAPS2, I believe that using compu...     4.0
2      Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...     3.0
3      Dear Local Newspaper, @CAPS1 I have found that...     5.0
4      Dear @LOCATION1, I know having computers has a...     4.0
...                                                  ...     ...
10214  There were many obstacles the builders of the ...     3.0
10215  Some of the problems with the constructing of ...     2.0
10216  The builders of the Empire State building face...     3.0
10217  The obstacles the builders of the Empire State...     1.0
10218  You want me to tell you what they had to go th...     2.0

[10218 rows x 2 columns]>

In [17]:
!pip install pyspellchecker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 46.8 MB/s eta 0:00:00


In [18]:
!pip install language_tool_python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.0 MB/s eta 0:00:00


In [30]:
!pip install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
!pip install transformers torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [39]:
!pip install -U numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 34.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.2
    Uninstalling numpy-1.23.2:
      Successfully uninstalled numpy-1.23.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.3 which is incompatible.
tensorflow-ranking 0.5.2 requires numpy==1.23.2, but you have numpy 1.24.3 which is incompatible.


In [40]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
   
data = pd.read_csv("/content/essay_dataset_modified_v2.csv")

In [43]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_and_encode(data, tokenizer):
    encodings = tokenizer(data["essay"].tolist(), truncation=True, padding=True, max_length=512)
    input_ids = encodings["input_ids"]
    attention_mask = encodings["attention_mask"]
    return input_ids, attention_mask

train_data = data.sample(frac=0.8, random_state=42)
val_data = data.drop(train_data.index)
train_data = train_data.dropna()
val_data = val_data.dropna()
train_encodings = tokenize_and_encode(train_data, tokenizer)
val_encodings = tokenize_and_encode(val_data, tokenizer)

In [44]:
class EssayDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __getitem__(self, idx):
        item = {
            "input_ids": torch.tensor(self.input_ids[idx]),
            "attention_mask": torch.tensor(self.attention_masks[idx]),
            "labels": torch.tensor(self.labels[idx]),
        }
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = EssayDataset(*tokenize_and_encode(train_data, tokenizer), train_data["scores"].tolist())
val_dataset = EssayDataset(*tokenize_and_encode(val_data, tokenizer), val_data["scores"].tolist())
